# Hyperparamter Tuning via Optuna

## Global Optuna

### Loading Libraries

In [7]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Warnings
import warnings

# Time
import time

# Notebook Optimizer
from tqdm.notebook import tqdm

# Scikit-Learn
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense, Dropout 
from tensorflow.keras import backend as K

# Optuna
import optuna

In [8]:
warnings.filterwarnings("ignore")

### Function Definition

#### Placing Model Architecture Function

In [12]:
def create_model(trial: optuna.trial.Trial, input_size: int): 
    model = Sequential()
    model.add(Dense(input_size,input_shape=(input_size,),activation='relu')) 

    num_layers = trial.suggest_int('num_layers',low=0,high=3)  
    for layer_i in range(num_layers):  
        n_units = trial.suggest_int(f'n_units_layer_{layer_i}',low=10,high=50,step=5)  
        dropout_rate = trial.suggest_float(f'dropout_rate_layer_{layer_i}',low=0,high=0.5)  
        actv_func = trial.suggest_categorical(f'actv_func_layer_{layer_i}',['relu','tanh','elu'])  

        model.add(Dropout(dropout_rate))  
        model.add(Dense(n_units,activation=actv_func)) 

    model.add(Dense(1,activation='sigmoid')) 
    return model 

#### Optimizers Function

In [13]:
def create_optimizer(trial: optuna.trial.Trial): 
	opt_kwargs = {} 
	opt_selected = trial.suggest_categorical('optimizer', ['Adam','SGD']) 
	if opt_selected == 'SGD': 
		opt_kwargs['lr'] = trial.suggest_float('sgd_lr',1e-5,1e-1,log=True) 
		opt_kwargs['momentum'] = trial.suggest_float('sgd_momentum',1e-5,1e-1,log=True) 
	else: #’Adam’ 
		opt_kwargs['lr'] = trial.suggest_float('adam_lr',1e-5,1e-1,log=True) 

	optimizer = getattr(tf.optimizers,opt_selected)(**opt_kwargs) 
	return optimizer

#### Custom Metric

In [14]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#### Preprocessing Function

In [15]:
def preprocessing(X: pd.DataFrame, 
                  numeric_preprocessor, categorical_preprocessor,
                  is_train = True
                 ):
    if is_train:
        X[numerical_feats] = numeric_preprocessor.fit_transform(X[numerical_feats])
        X_cat = categorical_preprocessor.fit_transform(X[categorical_feats]).toarray()
        X_cat = pd.DataFrame(X_cat,columns=categorical_preprocessor.get_feature_names_out())
        X = X.drop(columns=categorical_feats).reset_index(drop=True)
        X = pd.concat([X,X_cat],axis=1)
    else:
        X[numerical_feats] = numeric_preprocessor.transform(X[numerical_feats])
        X_cat = categorical_preprocessor.transform(X[categorical_feats]).toarray()
        X_cat = pd.DataFrame(X_cat,columns=categorical_preprocessor.get_feature_names_out())
        X = X.drop(columns=categorical_feats).reset_index(drop=True)
        X = pd.concat([X,X_cat],axis=1)
    
    return X, numeric_preprocessor, categorical_preprocessor

#### Training function

In [16]:
def train(trial, df_train: pd.DataFrame, df_val: pd.DataFrame = None, use_pruner: bool = False):
    X_train,y_train = df_train.drop(columns=['y']), df_train['y']
    
    if df_val is not None:
        X_val,y_val = df_val.drop(columns=['y']), df_val['y'] 

    #Preprocessing
    numeric_preprocessor = StandardScaler()
    categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
    
    X_train,numeric_preprocessor,categorical_preprocessor = preprocessing(X_train,
                                                                          numeric_preprocessor,
                                                                          categorical_preprocessor,
                                                                          is_train=True)
    if df_val is not None:
        X_val,_,_ = preprocessing(X_val,
                                  numeric_preprocessor,categorical_preprocessor,
                                  is_train=False)

    #Build model & optimizer
    model = create_model(trial,X_train.shape[1])
    optimizer = create_optimizer(trial)
    
    callbacks = []
    if use_pruner:
        callbacks.append(optuna.integration.TFKerasPruningCallback(trial,'val_f1_m'))

    model.compile(loss='binary_crossentropy',optimizer=optimizer,
                  metrics=[f1_m],
                 )
    history = model.fit(X_train,y_train,
                        epochs=trial.suggest_int('epoch',15,50),
                        batch_size=64,
                        validation_data=(X_val,y_val) if df_val is not None else None,
                        callbacks=callbacks,
                        verbose=False
                       )
    if df_val is not None:
        return np.mean(history.history['val_f1_m'])
    else:
        return model

#### The Objective Function

In [17]:
def objective(trial: optuna.trial.Trial, df_train: pd.DataFrame, use_pruner: bool = False): 
    #Split into Train and Validation data
    df_train_hp, df_val = train_test_split(df_train, test_size=0.1, random_state=0)
    
    # Train and Validate Model
    val_f1_score = train(trial, df_train_hp, df_val, use_pruner)
        
    return val_f1_score

#### Final Training & Evaluation Function: `To Test The Best Set of Hyperparmeters`

In [18]:
def train_and_evaluate_final(df_train: pd.DataFrame, df_test: pd.DataFrame, **kwargs):
    X_train,y_train = df_train.drop(columns=['y']), df_train['y']
    X_test,y_test = df_test.drop(columns=['y']), df_test['y'] 
    
    # Preprocessing
    numeric_preprocessor = StandardScaler()
    categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
    X_train,numeric_preprocessor,categorical_preprocessor = preprocessing(X_train,
                                                                          numeric_preprocessor,
                                                                          categorical_preprocessor,
                                                                          is_train=True)
    X_test,_,_ = preprocessing(X_test,numeric_preprocessor,categorical_preprocessor,
                              is_train=False)

    #Build model
    input_size = X_train.shape[1]
    model = Sequential()
    model.add(Dense(input_size,input_shape=(input_size,),activation='relu')) 

    num_layers = kwargs.get('num_layers',0)  
    for layer_i in range(num_layers):  
        n_units = kwargs.get(f'n_units_layer_{layer_i}',0)  
        dropout_rate = kwargs.get(f'dropout_rate_layer_{layer_i}',0)  
        actv_func = kwargs.get(f'actv_func_layer_{layer_i}','relu')  

        model.add(Dropout(dropout_rate))  
        model.add(Dense(n_units,activation=actv_func)) 

    model.add(Dense(1,activation='sigmoid'))
    
    #Build Optimizer
    opt_kwargs = {} 
    opt_selected = kwargs.get('optimizer', 'Adam')
    if opt_selected == 'SGD': 
        opt_kwargs['lr'] = kwargs.get('sgd_lr',1e-5) 
        opt_kwargs['momentum'] = kwargs.get('sgd_momentum',1e-5) 
    else: #’Adam’ 
        opt_kwargs['lr'] = kwargs.get('adam_lr',1e-5) 

    optimizer = getattr(tf.optimizers,opt_selected)(**opt_kwargs) 
    
    #Training process
    model.compile(loss='binary_crossentropy',optimizer=optimizer,
                  metrics=[f1_m],
                 )
    print(model.summary())
    history = model.fit(X_train,y_train,
                        epochs=kwargs.get('epoch',15),
                        batch_size=64,
                        validation_data=None,
                        verbose=True
                       )
    
    # Evaluation Process
    y_test_pred_probas = model.predict(X_test)
    y_test_pred = [1 if x[0]>0.5 else 0 for x in y_test_pred_probas]
    
    print("="*100)
    print("F1-Score on Test Data: ",f1_score(y_test, y_test_pred))